<b> TO BE DONE </b>

# MODEL-BASED META-LEARNING USING Conditional Neural Processes

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd drive/MyDrive/'Colab Notebooks'
#%cd meta-learning-course-notebooks/1_MAML/
#!ls

In [ ]:
#!pip install import_ipynb --quiet

In [ ]:
#!pip install learn2learn --quiet

In [1]:
import import_ipynb
import utils
import models
utils.hide_toggle('Imports 1')

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from models.ipynb


In [2]:
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from l2lutils import KShotLoader
from IPython import display
import torch.nn as nn
utils.hide_toggle('Imports 2')

importing Jupyter notebook from l2lutils.ipynb


# Data Generation/Loading

In [3]:
#Generate data - euclidean
meta_train_ds, meta_test_ds, full_loader = utils.euclideanDataset(n_samples=10000,n_features=20,n_classes=10,batch_size=32)

In [3]:
#Generate data - sinusoidal mix
meta_train_ds, meta_test_ds, full_loader = utils.sinDataset(n_samples=10000,length=20,n_classes=10,batch_size=16)

In [4]:
meta_train_ds, meta_test_ds, full_loader = utils.mnist_data(frac=0.2)

In [30]:
# Define an MLP network. Note that input dimension has to be data dimension. For classification
# final dimension has to be number of classes; for regression one.
#torch.manual_seed(10)
net = models.MLP(dims=[20,64,10])

In [31]:
print(net)

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=20, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=10, bias=True)
    (3): LogSoftmax(dim=1)
  )
)


In [27]:
# Train the network; note that network is trained in place so repeated calls further train it.
net,losses,accs=models.Train(net,full_loader,lr=1e-2,epochs=50,verbose=True)

Epoch   49 Loss: 2.88737e-02 Accuracy: 0.99402


In [12]:
#Training accuracy.
models.accuracy(net,meta_train_ds.samples,meta_train_ds.labels,verbose=True)

7461.0 7500


0.9948

In [13]:
# Test accuracy.
models.accuracy(net,meta_test_ds.samples,meta_test_ds.labels)

2437.0 2500


0.9748

# Meta-Learning: Tasks

Generate a k-shot n-way loader using the meta-training dataset

In [4]:
meta_train_kloader=KShotLoader(meta_train_ds,shots=5,ways=2)

Sample a task - each task has a k-shot n-way training set and a similar test set

In [12]:
d_train,d_test=meta_train_kloader.get_task()

Let's try directly learning using the task training set albeit its small size: create a dataset and loader and train it with the earlier network and Train function.

In [13]:
taskds = utils.MyDS(d_train[0],d_train[1])

In [14]:
d_train_loader = torch.utils.data.DataLoader(dataset=taskds,batch_size=1,shuffle=True)

In [15]:
net,losses,accs=models.Train(net,d_train_loader,lr=1e-1,epochs=10,verbose=True)

Epoch   9 Loss: 2.45026e-02 Accuracy: 1.00000


How does it do on the test set of the sampled task?

In [16]:
models.accuracy(net,d_test[0],d_test[1])

7.0 10


0.7

# CNP-based  Meta-learning

In [4]:
# optimisers from torch
import torch.optim as optim
import torch.nn.functional as F

In [5]:
lossfn = torch.nn.NLLLoss()

Conditional Neural Process Network

In [6]:
class CNP(nn.Module):
    def __init__(self,n_features=1,dims=[32,32],n_classes=2,lr=1e-4):
        super(CNP,self).__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        dimL1 = [n_features]+dims
        dimL2=[n_features+n_classes*dims[-1]]+dims+[n_classes]
        self.mlp1 = models.MLP(dims=dimL1,task='embedding')
        self.mlp2 = models.MLP(dims=dimL2)
        self.optimizer=torch.optim.Adam(self.parameters(),lr=lr)
    def adapt(self,X,y):
        R = self.mlp1(X)
        m = torch.eye(self.n_classes)[y].transpose(0,1)/self.n_classes
        r = (m@R).flatten().unsqueeze(0)
        #r = (R.sum(dim=0)/X.shape[0]).unsqueeze(0)
        return r
    def forward(self,Y,r):
        rr = r.repeat(Y.shape[0],1)
        p = self.mlp2(torch.cat((Y,rr),dim=1))
        return p
utils.hide_toggle('Class CNP')

Get a task dataset.

In [7]:
meta_train_kloader=KShotLoader(meta_train_ds,shots=5,ways=2,num_tasks=1000)

In [8]:
d_train,d_test = meta_train_kloader.get_task()

In [9]:
net = CNP(n_features=20,dims=[32,64,32])

In [11]:
print(net.mlp1,net.mlp2)

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=20, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
  )
) MLP(
  (layers): ModuleList(
    (0): Linear(in_features=84, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=2, bias=True)
    (7): LogSoftmax(dim=1)
  )
)


In [12]:
r = net.adapt(d_train[0],d_train[1])
r

tensor([[ 1.8370e-01, -3.3161e-01, -1.7857e-01,  1.8385e-01,  6.6111e-01,
          7.9923e-01, -2.8412e-01, -1.1213e+00, -2.8675e-01, -2.0983e-01,
          5.1299e-01, -1.1254e-02,  1.1964e+00,  1.2249e+00, -1.2897e+00,
         -1.2033e+00,  1.0958e+00, -2.4643e-01, -2.6621e-01,  7.2548e-02,
         -7.3273e-01,  2.1614e-01,  5.8698e-01, -2.6827e-01, -7.0521e-01,
          2.5810e-01,  5.0262e-01,  1.1917e+00,  1.1456e-01,  1.2058e-01,
          2.1144e+00, -1.0731e+00,  1.0130e-01, -9.8845e-02, -2.6045e-01,
         -1.4196e-01,  1.9594e-02,  6.9226e-01,  1.8853e-01, -2.5106e-01,
         -1.4936e-01, -1.9686e-01, -1.6853e-01, -2.0616e-01,  6.8305e-01,
          5.6174e-01, -3.8541e-01, -6.2113e-01,  4.7915e-01, -2.8740e-01,
          1.3566e-01,  6.1992e-01, -8.5800e-02, -2.6516e-01,  2.9519e-01,
         -1.2892e-03, -1.0783e-01,  1.7595e-01,  1.3833e-01,  9.8179e-01,
         -8.9019e-02,  5.4155e-02,  1.3350e+00, -6.4481e-01]],
       grad_fn=<UnsqueezeBackward0>)

In [13]:
net(d_train[0],r)

tensor([[-0.8264, -0.5756],
        [-0.8509, -0.5569],
        [-0.8394, -0.5655],
        [-0.8743, -0.5399],
        [-0.8439, -0.5622],
        [-0.8205, -0.5802],
        [-0.8328, -0.5706],
        [-0.7965, -0.5995],
        [-0.7794, -0.6138],
        [-0.7985, -0.5979]], grad_fn=<LogSoftmaxBackward>)

# Putting it all together: CNP-based Meta-learning
Now let's put it together in a loop - CNP model-based meta-learning algorithm:

In [14]:
# Redifning accuracy function so that it takes h - dataset context - as input since net requires it.
def accuracy(Net,X_test,y_test,h,verbose=True):
    #Net.eval()
    m = X_test.shape[0]
    y_pred = Net(X_test,h)
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_test).float().sum().item()
    if verbose: print(correct,m)
    accuracy = correct/m
    #Net.train()
    return accuracy

In [41]:
classes_train = [i for i in range(5)]
classes_test = [i+5 for i in range(5)]
classes_train, classes_test

([0, 1, 2, 3, 4], [5, 6, 7, 8, 9])

In [42]:
import learn2learn as l2l
import torch.optim as optim
shots,ways = 5,2
net = CNP(n_features=20,n_classes=ways,dims=[32,64,32],lr=1e-4)
lossfn = torch.nn.NLLLoss()
meta_train_kloader=KShotLoader(meta_train_ds,shots=shots,ways=ways,num_tasks=1000,classes=classes_train)

In [43]:
#Meta-testing task loader for later.
meta_test_kloader=KShotLoader(meta_test_ds,shots=shots,ways=ways,classes=classes_test)

In [46]:
epoch=0
n_epochs=100
task_count=50
while epoch<n_epochs:
    test_loss = 0.0
    test_acc = 0.0
    # Sample and train on a task
    for task in range(task_count):
        d_train,d_test=meta_test_kloader.get_task()
        rp = torch.randperm(d_train[1].shape[0])
        d_train0=d_train[0][rp]
        d_train1=d_train[1][rp]
        x_tr = d_train0
        d_tr = x_tr 
        h = net.adapt(d_tr,d_train1)
        rp1 = torch.randperm(d_test[1].shape[0])
        d_test0=d_test[0][rp1]
        d_test1=d_test[1][rp1]
        x_ts = d_test0
        y_ts_sh = torch.zeros(x_ts.shape[0],ways)
        d_ts = x_ts 
        test_preds = net(d_ts,h)
        train_preds = net(d_tr,h)
        # Accumulate losses over tasks - note train and test loss both included
        test_loss += lossfn(test_preds,d_test1)+lossfn(train_preds,d_train1)
        net.eval()
        test_acc += accuracy(net,d_ts,d_test1,h,verbose=False)
        net.train()
    #Update the network weights
    print('Epoch  % 2d Loss: %2.5e Avg Acc: %2.5f'%(epoch,test_loss/task_count,test_acc/task_count))
    display.clear_output(wait=True)
    net.optimizer.zero_grad()
    test_loss.backward()
    net.optimizer.step()
    epoch+=1
    

Epoch   99 Loss: 4.53195e-01 Avg Acc: 0.91000


Now test the trained CNP network and to tasks sampled from the meta_test_ds dataset:

In [48]:
meta_test_kloader=KShotLoader(meta_test_ds,shots=shots,ways=ways)
test_acc = 0.0
task_count = 50
adapt_steps = 1
# Sample and train on a task
for task in range(task_count):
    d_train,d_test=meta_test_kloader.get_task()
    x_tr = d_train[0]
    y_tr_sh = torch.cat((torch.zeros(1,ways),torch.eye(ways)[d_train[1][1:]]))
    d_tr = x_tr #torch.cat((x_tr,y_tr_sh),1)
    h=net.adapt(d_tr,d_train[1])
    x_ts = d_test[0]
    y_ts_sh = torch.zeros(x_ts.shape[0],ways)
    d_ts = x_ts #torch.cat((x_ts,y_ts_sh),1)
    test_preds = net(d_ts,h)
    test_acc += accuracy(net,d_ts,d_test[1],h,verbose=False)
    # Done with a task
net.train()
print('Avg Acc: %2.5f'%(test_acc/task_count))

Avg Acc: 0.75400
